# Initialization


## Module Imports


In [ ]:
import os
import random
import time
import tracemalloc
import warnings

from typing import Callable

import numpy as np
from numpy.typing import NDArray

import cv2
import matplotlib.pyplot as plt
import pandas as pd

from scipy.spatial.distance import directed_hausdorff
from sklearn.model_selection import ParameterGrid

In [ ]:
from ellipse_detection.ellipse_detector import (
    EllipseDetector as VFEllipseDetector,
)

from random_hough_ellipse_ori import (
    EllipseDetector as RHTEllipseDetector,
    EllipseDetectorInfo as RHTEllipseDetectorInfo,
)

from random_hough_ellipse_jit import compile_jit_functions, random_hough_ellipse

## Loading Functions


In [ ]:
def load_image_file(index: int, suffix: str) -> NDArray:
    for i in range(20):
        image = cv2.imread(f"data/{index}_{i or ''}{suffix}.png", cv2.IMREAD_GRAYSCALE)
        if image is not None:
            return image
    raise FileNotFoundError(f"{index} not found")


def load_image(index: int) -> NDArray:
    return load_image_file(index, "HC")


def load_mask(index: int) -> NDArray:
    return load_image_file(index, "HC_Annotation")


def load_image_mask(
    index: int,
) -> tuple[NDArray, NDArray]:
    return load_image(index), load_mask(index)


def load_images_masks(indices: list[int]) -> tuple[list[NDArray], list[NDArray]]:
    images = [load_image(index) for index in indices]
    masks = [load_mask(index) for index in indices]
    return images, masks

In [ ]:
def get_valid_indices() -> list[int]:
    valid_indices = []
    filenames = os.listdir("data")
    image_indices = list(
        map(
            lambda x: int(x.split("_")[0]),
            filter(lambda x: x.endswith("HC.png"), filenames),
        )
    )
    mask_indices = list(
        map(
            lambda x: int(x.split("_")[0]),
            filter(lambda x: x.endswith("HC_Annotation.png"), filenames),
        )
    )
    for i in range(1, 1000):
        if i in image_indices and i in mask_indices:
            valid_indices.append(i)
    return valid_indices

In [ ]:
def get_truth_data() -> pd.DataFrame:
    raw_truth_data = pd.read_csv("data.csv")
    truth_data = pd.DataFrame()
    truth_data["index"] = raw_truth_data["filename"].str.split("_").str[0].astype(int)
    truth_data["pixel_size"] = raw_truth_data["pixel size"]
    truth_data["circumference"] = raw_truth_data["head circumference (mm)"]
    truth_data.set_index("index", inplace=True)
    return truth_data

## Plotting Functions


In [ ]:
def plot_multiple_images(
    image_data: list[tuple[NDArray, str]] | list[NDArray], n_cols=6
):
    if not isinstance(image_data[0], tuple):
        img_data = [(image, "") for image in image_data]
    else:
        img_data = image_data
    n_rows = int(np.ceil(len(img_data) / n_cols))
    fig, axes = plt.subplots(
        n_rows, n_cols, figsize=(int(n_cols * 4.5), int(n_rows * 3))
    )
    axes = axes.flatten()
    fig.tight_layout(pad=0)
    fig.patch.set_facecolor("dimgray")  # type: ignore
    for ax in axes:
        ax.axis("off")
    for i, (image, title) in enumerate(img_data):
        axes[i].imshow(image, cmap="gray")
        axes[i].set_title(title)
    plt.show()

In [ ]:
def draw_ellipse(
    image: NDArray,
    center: tuple[float, float],
    axes: tuple[float, float],
    angle: float,
    idx: int,
):
    cv2.ellipse(
        image,
        (int(center[0]), int(center[1])),
        (int(axes[0]), int(axes[1])),
        angle * 180 / np.pi,
        0,
        360,
        color=(255 * (idx == 2), 255 * (idx == 0), 255 * (idx == 1)),
        thickness=2 if idx == 0 else 1,
    )

## Data-related Functions


In [ ]:
def create_border_mask(shape: tuple[int, int], border: int) -> NDArray[np.bool8]:
    mask = np.zeros(shape, dtype=np.bool8)
    mask[:border, :] = mask[-border:, :] = mask[:, :border] = mask[:, -border:] = 1
    return mask


def update_image_value(image: NDArray, mask: NDArray[np.bool8], value: int) -> NDArray:
    new_image = image.copy()
    new_image[mask] = value
    return new_image


def downscale_image(image: NDArray, scale: float = 0.5) -> NDArray:
    return cv2.resize(image, (0, 0), fx=scale, fy=scale)


def quantile(arr: NDArray | list, q: float) -> float:
    return np.quantile(arr, q).item()

# Exploration


## Load and Sample


In [ ]:
valid_indices = get_valid_indices()

random.seed(1)
sample_indices = random.choices(valid_indices, k=9)

sample_images, sample_masks = load_images_masks(sample_indices)


def plot_multiple_images_with_original(images: list[NDArray]):
    plot_multiple_images([img for imgs in zip(sample_images, images) for img in imgs])

## Simple Processing


### Resize


In [ ]:
resized_images = list(map(lambda x: downscale_image(x), sample_images))
print(resized_images[0].shape)

### Blur


In [ ]:
blurred_images = list(map(lambda x: cv2.medianBlur(x, 11), resized_images))

In [ ]:
# plot_multiple_images_with_original(blurred_images)

## Edge Detection


### Unused Edges


In [ ]:
unused_edges = list(
    map(
        lambda x: (
            cv2.dilate(
                cv2.Canny(update_image_value(x, x != 0, 255), 1, 1, None, 3),
                np.ones((3, 3), np.uint8),
            )
            == 255
        )
        | create_border_mask((x.shape[0], x.shape[1]), border=5),
        blurred_images,
    )
)

### Main Edges


In [ ]:
main_edges = list(
    map(
        lambda x: update_image_value(
            cv2.Canny(x[0], quantile(x[0], 0.9), quantile(x[0], 0.95), None, 3),
            x[1],
            0,
        ),
        zip(blurred_images, unused_edges),
    )
)

In [ ]:
# plot_multiple_images_with_original(main_edges)

## Ellipse Detection


### Very Fast Ellipse Detection

https://github.com/horiken4/ellipse-detection


In [ ]:
def very_fast_ellipse_detection_and_draw(
    image: NDArray, image_edge: NDArray | None = None
) -> NDArray:
    ellipse_detector = VFEllipseDetector()
    ellipses = ellipse_detector.detect(image, image_edge)

    print()
    result = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    for i, ellipse in enumerate(ellipses):
        if i == 3:
            break
        print(ellipse.accuracy_score, end=", ")
        draw_ellipse(
            result,
            ellipse.center,
            (ellipse.major_len, ellipse.minor_len),
            ellipse.angle,
            i,
        )
    return result


ellipse_images = list(
    map(very_fast_ellipse_detection_and_draw, blurred_images, main_edges)
)

In [ ]:
plot_multiple_images_with_original(ellipse_images)

### Random Hough Ellipse - Original

https://github.com/Po-Ting-lin/RandomizedHoughEllipseDetector


In [ ]:
ed_info = RHTEllipseDetectorInfo(
    MaxIter=1000,
    LineFittingArea=7,
    MajorAxisBound=(80, 350),
    MinorAxisBound=(80, 350),
    MaxFlattening=0.6,
    SimilarCenterDist=5,
    SimilarMajorAxisDist=10,
    SimilarMinorAxisDist=10,
    SimilarAngleDist=np.pi / 18,
)


def random_hough_ellipse_ori_and_draw(edge: NDArray) -> NDArray:
    ellipse_detector = RHTEllipseDetector(ed_info, edge)
    best_candidates = ellipse_detector.run()

    print()
    result = cv2.cvtColor(edge, cv2.COLOR_GRAY2RGB)
    for i, cand in enumerate(best_candidates):
        print(cand.score, end=", ")
        draw_ellipse(result, (cand.p, cand.q), (cand.a, cand.b), cand.angle, i)
    return result


ellipse_images = list(map(random_hough_ellipse_ori_and_draw, main_edges))

In [ ]:
plot_multiple_images_with_original(ellipse_images)

### Random Hough Ellipse - Optimized


In [ ]:
compile_jit_functions()

In [ ]:
def random_hough_ellipse_optimized_and_draw(edge: NDArray) -> NDArray:
    best_candidates = random_hough_ellipse(
        edge,
        1000,
        edge,
        7,
        20,
        10,
        10,
        np.pi / 18,
        (80, 350),
        (80, 350),
        0.6,
        100000,
        1000,
        3,
        100,
    )

    print()
    result = cv2.cvtColor(edge, cv2.COLOR_GRAY2RGB)
    for i, cand in enumerate(best_candidates):
        print(cand[0], end=", ")
        draw_ellipse(result, (cand[1], cand[2]), (cand[3], cand[4]), cand[5], i)
    return result


ellipse_images = list(map(random_hough_ellipse_optimized_and_draw, main_edges))

In [ ]:
plot_multiple_images_with_original(ellipse_images)

# Evaluation


## Functions


### Ellipse


In [ ]:
Ellipse = tuple[float, float, float, float, float]

In [ ]:
def edge_ellipse_from_params(
    ellipse: Ellipse, mask_shape: tuple[int, int] = (500, 500)
) -> NDArray:
    edge = np.zeros(mask_shape, dtype=np.uint8)
    cv2.ellipse(
        edge,
        (int(ellipse[0]), int(ellipse[1])),
        (int(ellipse[2]), int(ellipse[3])),
        ellipse[4] * 180 / np.pi,
        0,
        360,
        color=(1,),
        thickness=1,
    )
    return edge == 1

In [ ]:
def fill_ellipse_from_params(
    ellipse: Ellipse, mask_shape: tuple[int, int] = (500, 500)
) -> NDArray:
    x = np.arange(0, mask_shape[1])
    y = np.arange(0, mask_shape[0]).reshape(-1, 1)
    term_1 = (
        (x - ellipse[0]) * np.cos(ellipse[4]) + (y - ellipse[1]) * np.sin(ellipse[4])
    ) ** 2
    term_2 = (
        (x - ellipse[0]) * np.sin(ellipse[4]) - (y - ellipse[1]) * np.cos(ellipse[4])
    ) ** 2
    combined_term = (term_1 / ellipse[2] ** 2) + (term_2 / ellipse[3] ** 2)
    ellipse = combined_term <= 1
    return ellipse  # type: ignore

In [ ]:
def fill_ellipse_from_edge(edge: NDArray) -> NDArray:
    out_fill = edge.copy()
    cv2.floodFill(
        out_fill,
        np.zeros((out_fill.shape[0] + 2, out_fill.shape[1] + 2), np.uint8),
        (0, 0),
        1,  # type: ignore
    )
    in_fill = (edge == 1) | (out_fill == 0)
    return in_fill

In [ ]:
def ellipse_circumference(
    ellipse: Ellipse, pixel_size: float, scale: float = 0.5
) -> float:
    # Ramanujan approximation
    a, b = ellipse[2], ellipse[3]
    return (
        np.pi
        * (3 * (a + b) - np.sqrt((3 * a + b) * (a + 3 * b)))
        * pixel_size
        * (1 / scale)
    )

### Segmentation Metrics


In [ ]:
def hausdorff_distance(
    true: NDArray, pred: NDArray, pixel_size: float, scale: float = 0.5
) -> float:
    return (
        directed_hausdorff(np.argwhere(true), np.argwhere(pred))[0]
        * pixel_size
        * (1 / scale)
    )

In [ ]:
def iou(true: NDArray, pred: NDArray) -> float:
    intersection = (true & pred).sum()
    union = (true | pred).sum()
    return intersection / union

In [ ]:
def iou_to_dice(iou: float) -> float:
    return 2 * iou / (1 + iou)

### Batch Evaluation


In [ ]:
def evaluate_ellipse_detections(
    truth_data: pd.DataFrame,
    true_ellipse_masks: list[NDArray],
    pred_ellipse_params: list[Ellipse],
    valid_indices: list[int],
) -> pd.DataFrame:
    result = []
    for i, index in enumerate(valid_indices):
        pixel_size = truth_data.loc[index, "pixel_size"].item()  # type: ignore

        true_mask = downscale_image(true_ellipse_masks[i])
        mask_shape = (true_mask.shape[0], true_mask.shape[1])
        pred_mask = edge_ellipse_from_params(pred_ellipse_params[i], mask_shape)
        hd_value = hausdorff_distance(true_mask, pred_mask, pixel_size)

        true_mask_filled = fill_ellipse_from_edge(true_mask)
        pred_mask_filled = fill_ellipse_from_params(pred_ellipse_params[i], mask_shape)
        dice_score = iou_to_dice(iou(true_mask_filled, pred_mask_filled))

        true_circumference = truth_data.loc[index, "circumference"].item()  # type: ignore
        pred_circumference = ellipse_circumference(pred_ellipse_params[i], pixel_size)

        result.append(
            {
                "index": index,
                "hd": hd_value,
                "dice": dice_score,
                "diff": true_circumference - pred_circumference,
                "adiff": abs(true_circumference - pred_circumference),
            }
        )
    return pd.DataFrame(result)

### Helper


In [ ]:
def move_df_column(df: pd.DataFrame, old: int = -1, new: int = 0) -> pd.DataFrame:
    cols = df.columns.to_list()
    cols.insert(new, cols.pop(old))
    return df[cols]

## Processing Functions


### Very Fast Ellipse Detection


In [ ]:
def very_fast_ellipse_detection(
    images: list[NDArray],
    blur_k_size: int,
    unused_border: int,
    canny_quantile_1: float,
    canny_quantile_2: float,
    gaussian_k_size: int,
    gaussian_sigma_x: int,
    ellipse_straight_ratio_threshold: int,
    ellipse_center_reject_distance: int,
    ellipse_lying_threshold: float,
    ellipse_identify_threshold: float,
) -> list[Ellipse]:
    ellipse_list: list[Ellipse] = []
    for img in images:
        resized = downscale_image(img)
        blurred = cv2.medianBlur(resized, blur_k_size)
        unused_edge = (
            cv2.dilate(
                cv2.Canny(
                    update_image_value(blurred, blurred != 0, 255), 1, 1, None, 3
                ),
                np.ones((3, 3), np.uint8),
            )
            == 255
        )
        unused_edge = unused_edge | create_border_mask(
            (blurred.shape[0], blurred.shape[1]), border=unused_border
        )
        main_edge = update_image_value(
            cv2.Canny(
                blurred,
                quantile(blurred, canny_quantile_1),
                quantile(blurred, canny_quantile_2),
                None,
                3,
            ),
            unused_edge,
            0,
        )
        ellipse_detector = VFEllipseDetector(
            gaussian_k_size=(gaussian_k_size, gaussian_k_size),
            gaussian_sigma_x=gaussian_sigma_x,
            straight_ratio_threshold=ellipse_straight_ratio_threshold,
            ellipse_center_reject_distance=ellipse_center_reject_distance,
            lying_threshold=ellipse_lying_threshold,
            identify_threshold=ellipse_identify_threshold,
        )
        try:
            ellipse = ellipse_detector.detect(blurred, main_edge)[0]
        except Exception:
            ellipse_list.append((0, 0, 0, 0, 0))
            continue
        ellipse_list.append(
            (
                ellipse.center[0],
                ellipse.center[1],
                ellipse.major_len,
                ellipse.minor_len,
                ellipse.angle,
            )
        )
    return ellipse_list

### Random Hough Ellipse - Original


In [ ]:
def random_hough_ellipse_ori(
    images: list[NDArray],
    blur_k_size: int,
    unused_border: int,
    canny_quantile_1: float,
    canny_quantile_2: float,
    ellipse_max_iter: int,
    ellipse_line_fitting_area: int,
    ellipse_similar_center_dist: int,
    ellipse_similar_axis_dist: int,
    ellipse_similar_angle_dist: float,
) -> list[Ellipse]:
    ellipse_list: list[Ellipse] = []
    for img in images:
        resized = downscale_image(img)
        blurred = cv2.medianBlur(resized, blur_k_size)
        unused_edge = (
            cv2.dilate(
                cv2.Canny(
                    update_image_value(blurred, blurred != 0, 255), 1, 1, None, 3
                ),
                np.ones((3, 3), np.uint8),
            )
            == 255
        )
        unused_edge = unused_edge | create_border_mask(
            (blurred.shape[0], blurred.shape[1]), border=unused_border
        )
        main_edge = update_image_value(
            cv2.Canny(
                blurred,
                quantile(blurred, canny_quantile_1),
                quantile(blurred, canny_quantile_2),
                None,
                3,
            ),
            unused_edge,
            0,
        )
        ellipse_detector_info = RHTEllipseDetectorInfo(
            MaxIter=ellipse_max_iter,
            LineFittingArea=ellipse_line_fitting_area,
            MajorAxisBound=(80, 350),
            MinorAxisBound=(80, 350),
            MaxFlattening=0.6,
            SimilarCenterDist=ellipse_similar_center_dist,
            SimilarMajorAxisDist=ellipse_similar_axis_dist,
            SimilarMinorAxisDist=ellipse_similar_axis_dist,
            SimilarAngleDist=ellipse_similar_angle_dist,
        )
        ellipse_detector = RHTEllipseDetector(ellipse_detector_info, main_edge)
        ellipse = ellipse_detector.run()[0]
        ellipse_list.append((ellipse.p, ellipse.q, ellipse.a, ellipse.b, ellipse.angle))
    return ellipse_list

### Random Hough Ellipse - Optimized


In [ ]:
def random_hough_ellipse_optimized(
    images: list[NDArray],
    blur_k_size: int,
    unused_border: int,
    canny_quantile_1: float,
    canny_quantile_2: float,
    ellipse_max_iter: int,
    ellipse_line_fitting_area: int,
    ellipse_similar_center_dist: int,
    ellipse_similar_axis_dist: int,
    ellipse_similar_angle_dist: float,
) -> list[Ellipse]:
    ellipse_list: list[Ellipse] = []
    for img in images:
        resized = downscale_image(img)
        blurred = cv2.medianBlur(resized, blur_k_size)
        unused_edge = (
            cv2.dilate(
                cv2.Canny(
                    update_image_value(blurred, blurred != 0, 255), 1, 1, None, 3
                ),
                np.ones((3, 3), np.uint8),
            )
            == 255
        )
        unused_edge = unused_edge | create_border_mask(
            (blurred.shape[0], blurred.shape[1]), border=unused_border
        )
        main_edge = update_image_value(
            cv2.Canny(
                blurred,
                quantile(blurred, canny_quantile_1),
                quantile(blurred, canny_quantile_2),
                None,
                3,
            ),
            unused_edge,
            0,
        )
        ellipse = random_hough_ellipse(
            main_edge,
            ellipse_max_iter,
            main_edge,
            ellipse_line_fitting_area,
            ellipse_similar_center_dist,
            ellipse_similar_axis_dist,
            ellipse_similar_axis_dist,
            ellipse_similar_angle_dist,
            (80, 350),
            (80, 350),
            0.6,
            1000,
            2000,
            1,
            100,
        )[0]
        ellipse_list.append(
            (ellipse[1], ellipse[2], ellipse[3], ellipse[4], ellipse[5])
        )
    return ellipse_list

## Evaluation Process


In [ ]:
truth_data = get_truth_data()
truth_data

In [ ]:
valid_indices = get_valid_indices()
len(valid_indices)

In [ ]:
random.seed(0)
random.shuffle(valid_indices)

# test_indices = valid_indices[:200]
tune_indices = valid_indices[:]

# test_images, test_masks = load_images_masks(test_indices)
tune_images, tune_masks = load_images_masks(tune_indices)

In [ ]:
def detect_and_evaluate(
    detection_func: Callable[..., list[Ellipse]],
    parameter_grid: ParameterGrid,
    name: str,
    trial_slice: slice,
):
    random.seed(0)
    len_pg = len(parameter_grid)
    for i in random.sample(range(len_pg), len_pg)[trial_slice]:
        kwargs = parameter_grid[i]

        start_time = time.perf_counter()
        tune_ellipses = detection_func(tune_images, **kwargs)
        end_time = time.perf_counter()

        result_df = evaluate_ellipse_detections(
            truth_data, tune_masks, tune_ellipses, tune_indices
        )
        result_df["kwargs_index"] = i
        result_df = move_df_column(result_df)

        summary_df = pd.DataFrame(
            [
                {"index": i}
                | kwargs
                | {
                    "num_data": len(tune_indices),
                    "duration_mean": (end_time - start_time) / len(tune_indices),
                    "hd_mean": result_df["hd"].mean(),
                    "hd_std": result_df["hd"].std(),
                    "dice_mean": result_df["dice"].mean(),
                    "dice_std": result_df["dice"].std(),
                    "diff_mean": result_df["diff"].mean(),
                    "diff_std": result_df["diff"].std(),
                    "adiff_mean": result_df["adiff"].mean(),
                    "adiff_std": result_df["adiff"].std(),
                }
            ]
        )
        summary_df.set_index("index", inplace=True)

        result_file = f"{name}.csv"
        if not os.path.isfile(result_file):
            result_df.to_csv(result_file, index=False)
        else:
            result_df.to_csv(result_file, mode="a", header=False, index=False)

        summary_file = f"{name}_summary.csv"
        if not os.path.isfile(summary_file):
            summary_df.to_csv(summary_file)
        else:
            summary_df.to_csv(summary_file, mode="a", header=False)

In [ ]:
# basic_params = {
#     "blur_k_size": [5, 11, 17],
#     "unused_border": [5],
#     "canny_quantile_1": [0.7, 0.8, 0.9],
#     "canny_quantile_2": [0.9, 0.95],
# }

# vfed_param_grid = ParameterGrid(
#     basic_params
#     | {
#         "gaussian_k_size": [5, 11],
#         "gaussian_sigma_x": [3, 5],
#         "ellipse_straight_ratio_threshold": [10, 30, 50],
#         "ellipse_center_reject_distance": [10, 30, 50, 70],
#         "ellipse_lying_threshold": [0.1, 0.5],
#         "ellipse_identify_threshold": [0.1, 0.3, 0.5],
#     }
# )

# rhe_param_grid = ParameterGrid(
#     basic_params
#     | {
#         "ellipse_max_iter": [1000, 5000, 10000],
#         "ellipse_line_fitting_area": [7, 15],
#         "ellipse_similar_center_dist": [10, 20],
#         "ellipse_similar_axis_dist": [10, 20],
#         "ellipse_similar_angle_dist": [np.pi / 18, np.pi / 9],
#     }
# )

vfed_param_grid = ParameterGrid(
    {
        "blur_k_size": [11],
        "unused_border": [5],
        "canny_quantile_1": [0.7, 0.8],
        "canny_quantile_2": [0.9, 0.95],
        "gaussian_k_size": [5, 11],
        "gaussian_sigma_x": [3],
        "ellipse_straight_ratio_threshold": [30, 50],
        "ellipse_center_reject_distance": [70],
        "ellipse_lying_threshold": [0.1, 0.5],
        "ellipse_identify_threshold": [0.5],
    }
)

rhe_param_grid = ParameterGrid(
    {
        "blur_k_size": [11],
        "unused_border": [5],
        "canny_quantile_1": [0.8, 0.9],
        "canny_quantile_2": [0.9, 0.95],
        "ellipse_max_iter": [5000, 10000],
        "ellipse_line_fitting_area": [7],
        "ellipse_similar_center_dist": [20],
        "ellipse_similar_axis_dist": [10],
        "ellipse_similar_angle_dist": [np.pi / 18, np.pi / 9],
    }
)

In [ ]:
warnings.filterwarnings("ignore")
detect_and_evaluate(very_fast_ellipse_detection, vfed_param_grid, "vfed", slice(0, 50))
warnings.filterwarnings("default")

# 982m

In [ ]:
detect_and_evaluate(random_hough_ellipse_ori, rhe_param_grid, "rhe_ori", slice(0, 50))

# 767m

In [ ]:
detect_and_evaluate(
    random_hough_ellipse_optimized, rhe_param_grid, "rhe_opt", slice(0, 50)
)

# 120m

## Time and Memory Evaluation


In [ ]:
iters = [100, 1000, 5000, 10000, 15000, 20000, 25000, 30000]

kwargs = {
    "unused_border": 5,
    "ellipse_similar_center_dist": 20,
    "ellipse_similar_axis_dist": 10,
    "ellipse_similar_angle_dist": 0.349,
    "ellipse_line_fitting_area": 7,
    "canny_quantile_2": 0.95,
    "canny_quantile_1": 0.9,
    "blur_k_size": 11,
}

In [ ]:
start = time.perf_counter()

compile_jit_functions()

time.perf_counter() - start

In [ ]:
ori_times = []
for iter in iters:
    start_time = time.perf_counter()
    tune_ellipses = random_hough_ellipse_ori(
        tune_images[:10], **(kwargs | {"ellipse_max_iter": iter})
    )
    ori_times.append((time.perf_counter() - start_time) / 10)

opt_times = []
for iter in iters:
    start_time = time.perf_counter()
    tune_ellipses = random_hough_ellipse_optimized(
        tune_images[:10], **(kwargs | {"ellipse_max_iter": iter})
    )
    opt_times.append((time.perf_counter() - start_time) / 10)

print(ori_times)
print()
print(opt_times)

In [ ]:
ori_times = [
    0.09155316001269967,
    0.5396712900139391,
    2.6480242100078613,
    5.029666600003838,
    7.610079020005651,
    10.35959749999456,
    12.985094760009087,
    15.507335970015266,
]

opt_times = [
    0.034731719992123544,
    0.08878443001303822,
    0.29274345000740143,
    0.5003274300135672,
    0.6490047400118784,
    0.6741446000058204,
    0.6686791900079697,
    0.6703175900038332,
]


In [ ]:
tracemalloc.start()

In [ ]:
tracemalloc.reset_peak()
start, _ = tracemalloc.get_traced_memory()

compile_jit_functions()

end, _ = tracemalloc.get_traced_memory()
end - start

In [ ]:
ori_memories = []

random_hough_ellipse_ori(tune_images[:1], **(kwargs | {"ellipse_max_iter": 1000}))

for iter in iters:
    memory_usages = []
    kwargs["ellipse_max_iter"] = iter

    for i in range(10):
        tracemalloc.reset_peak()
        start, _ = tracemalloc.get_traced_memory()

        random_hough_ellipse_ori(
            tune_images[i : i + 1], **(kwargs | {"ellipse_max_iter": iter})
        )

        _, end = tracemalloc.get_traced_memory()
        memory_usages.append(end - start)

    ori_memories.append(sum(memory_usages) / len(memory_usages) / 1000)

ori_memories

In [ ]:
opt_memories = []

random_hough_ellipse_optimized(tune_images[:1], **(kwargs | {"ellipse_max_iter": 1000}))

for iter in iters:
    memory_usages = []

    for i in range(10):
        tracemalloc.reset_peak()
        start, _ = tracemalloc.get_traced_memory()

        random_hough_ellipse_optimized(
            tune_images[i : i + 1], **(kwargs | {"ellipse_max_iter": iter})
        )

        _, end = tracemalloc.get_traced_memory()
        memory_usages.append(end - start)

    opt_memories.append(sum(memory_usages) / len(memory_usages) / 1000)

opt_memories

In [ ]:
ori_memories = [
    1492.0021000000002,
    1506.0023999999999,
    1539.8454,
    1571.0932,
    1593.6758,
    1614.0096,
    1629.643,
    1644.7078000000001,
]

opt_memories = [
    757.0189399999999,
    756.9784000000001,
    756.97054,
    756.98364,
    756.98682,
    756.98492,
    756.97054,
    756.97054,
]


In [ ]:
fig, ax1 = plt.subplots()

color = "blue"
ax1.set_xlabel("iterations")
ax1.set_ylabel("time (s)", color=color)
ax1.set_ylim(0, 18)
ax1.plot(iters, ori_times, "ko--", iters, opt_times, "ko-")
ax1.tick_params(axis="y", labelcolor=color)

ax2 = ax1.twinx()

color = "red"
ax2.set_ylabel("memory (kB)", color=color)
ax2.set_ylim(0, 1800)
ax2.plot(iters, ori_memories, "ro--", iters, opt_memories, "ro-")
ax2.tick_params(axis="y", labelcolor=color)

fig.tight_layout()
plt.show()

# Other


## Ellipse Out of Mask


In [ ]:
# import time

# import cv2
# import numpy as np
# from numba import njit

# mask_shape = (270, 400)
# center = (225, 155)
# axis = (111, 88)
# angle = 0.253

# num_iter = 0


# def ellipse_out_of_mask_cv(
#     mask_shape: tuple[int, int],
#     center: tuple[int, int],
#     axis: tuple[int, int],
#     angle: float,
# ):
#     larger_shape = (mask_shape[0] + 2, mask_shape[1] + 2)
#     ref_mask = np.ones(larger_shape, dtype=np.uint8)
#     ref_mask[0, :] = ref_mask[-1, :] = ref_mask[:, 0] = ref_mask[:, -1] = 0

#     ellipse, out_of_mask = np.zeros_like(ref_mask), np.zeros_like(ref_mask)
#     cv2.ellipse(
#         ellipse, center, axis, angle * 180 / np.pi, 0, 360, color=(255,), thickness=1
#     )

#     out_of_mask[(ellipse == 255) & (ref_mask == 0)] = 1

#     return np.sum(out_of_mask) > 0


# print("CV2 Check", ellipse_out_of_mask_cv(mask_shape, center, axis, angle))


# start = time.perf_counter()
# for _ in range(num_iter):
#     ellipse_out_of_mask_cv(mask_shape, center, axis, angle)
# print("CV2 Time", time.perf_counter() - start)


# @njit
# def ellipse_out_of_mask(
#     mask_shape: tuple[int, int],
#     center: tuple[int, int],
#     axis: tuple[int, int],
#     angle: float,
# ):
#     larger_shape = (mask_shape[0] + 1, mask_shape[1] + 1)
#     ref_mask = np.zeros(larger_shape, dtype=np.uint8)
#     ref_mask[0, :] = ref_mask[-1, :] = ref_mask[:, 0] = ref_mask[:, -1] = 1

#     x = np.arange(0, larger_shape[0])
#     y = np.arange(0, larger_shape[1]).reshape(-1, 1)
#     term_1 = (
#         ref_mask
#         * ((x - center[0]) * np.cos(angle) + (y - center[1]) * np.sin(angle)) ** 2
#     )
#     term_2 = (
#         ref_mask
#         * ((x - center[0]) * np.sin(angle) - (y - center[1]) * np.cos(angle)) ** 2
#     )
#     combined_term = (term_1 / axis[0] ** 2) + (term_2 / axis[1] ** 2)
#     ellipse = (combined_term != 0) & (combined_term <= 1)

#     return np.count_nonzero(ellipse) > 0


# start = time.perf_counter()
# print("NumPy Check", ellipse_out_of_mask(mask_shape, center, axis, angle))
# print("NumPy Compile Time", time.perf_counter() - start)

# start = time.perf_counter()
# for _ in range(num_iter):
#     ellipse_out_of_mask(mask_shape, center, axis, angle)
# print("NumPy Time", time.perf_counter() - start)


# @njit
# def ellipse_out_of_mask_fast(
#     mask_shape: tuple[int, int],
#     center: tuple[int, int],
#     axis: tuple[int, int],
#     angle: float,
#     num_points: int = 50,
# ):
#     thetas = np.linspace(0, 2 * np.pi, num_points) - angle
#     tan_thetas = np.tan(thetas)

#     x_values = (
#         np.where((thetas > np.pi / 2) & (thetas < 3 * np.pi / 2), -1, 1)
#         * axis[0]
#         * axis[1]
#         / (np.sqrt(axis[1] ** 2 + (axis[0] ** 2) * np.square(tan_thetas)))
#     )

#     rotation_matrix = np.array(
#         [[-np.sin(angle), np.cos(angle)], [np.cos(angle), np.sin(angle)]]
#     )

#     coordinates = np.dot(
#         rotation_matrix, np.vstack((x_values * tan_thetas, x_values))
#     ) + np.array(center).reshape(-1, 1)

#     return np.any(
#         (coordinates < -1)
#         | (coordinates > np.array((mask_shape[1], mask_shape[0])).reshape(-1, 1))
#     )


# start = time.perf_counter()
# print(
#     "NumPy-Fast Check",
#     ellipse_out_of_mask_fast(mask_shape, center, axis, angle),
# )
# print("NumPy-Fast Compile Time", time.perf_counter() - start)

# start = time.perf_counter()
# for _ in range(num_iter):
#     ellipse_out_of_mask_fast(mask_shape, center, axis, angle)
# print("NumPy-Fast Time", time.perf_counter() - start)



## Thresholding


In [ ]:
# hist = cv2.calcHist(
#     [blurred_images[8]],
#     [0],
#     (blurred_images[8] > 40).astype(np.uint8),  # type: ignore
#     [256],
#     [0, 256],
# )
# plt.plot(hist)



In [ ]:
# thresholded_images = list(
#     map(
#         lambda x: cv2.adaptiveThreshold(
#             x, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 401, -40
#         ),
#         blurred_images,
#     )
# )



In [ ]:
# def dynamic_threshold(image: NDArray) -> NDArray:
#     count_ref = np.count_nonzero(image) * 0.2
#     threshold_ref = np.sort(np.array(image).flatten())[-int(count_ref / 2)]
#     return np.where(image > threshold_ref, image, 0)


# thresholded_images = list(map(dynamic_threshold, blurred_images))



## Hough Circle


In [ ]:
# def hough_circle_and_draw(image: NDArray) -> NDArray:
#     circles = cv2.HoughCircles(
#         image, cv2.HOUGH_GRADIENT, 1, 10, param1=50, param2=50, minRadius=30
#     )
#     image_circle = image.copy()
#     if circles is not None:
#         circles = np.around(circles).astype(np.uint16)  # type: ignore
#         for i, circle in enumerate(circles[0, :]):
#             if i == 5:
#                 break
#             cv2.circle(
#                 image_circle, (circle[0], circle[1]), circle[2], (255, 255, 255), 1
#             )
#     return image_circle


# circle_images = list(map(hough_circle_and_draw, blurred_images[:1]))